In [1]:
% cd C:/Users/lcvri/Downloads/pve_parser

C:\Users\lcvri\Downloads\pve_parser


In [2]:
import json
import pandas as pd
pd.set_option("display.max_colwidth", None)
import camelot
import pve_parser

In [8]:
path = 'programma-van-eisen-hoger-onderwijs.pdf'
# tables_obo  = camelot.read_pdf(path, pages='168-174', strip_text='\n')
# tables_obov = camelot.read_pdf(path, pages='175-185', strip_text='\n')
tables_anbs = camelot.read_pdf(path, pages='188-196', strip_text='\n')

In [4]:
eig_obo = dict(
    encoding = "latin1",
    separator = "|",
    date_format = "%Y%m%d"
)
eig_anbs = dict(
    encoding = "utf8",
    separator = "|",
    date_format = "%Y%m%d"
)

tablenames_obo = dict(
    VLP = "Voorlooprecord",
    PER = "Persoongegevens",
    NAT = "Nationaliteitsgegevens",
    VBT = "Verblijfstitelgegevens",
    VBV = "Verblijfsvergunninggegevens",
    ISG = "Inschrijvings-(deelname-)gegevens",
    GRD = "Graad-(resultaat-)gegevens",
    CTR = "Controletotalen",
)
tablenames_obov = dict(
    VLP = "Voorlooprecord",
    PER = "Wijziging in persoonsgegevens",
    NAT = "Wijziging in de nationaliteit",
    VBT = "Wijziging in de verblijfstitel",
    VBV = "Wijziging in de verblijfvergunning",
    ISG = "Wijziging in inschrijvings-(deelname-)gegevens",
    GRD = "Wijziging in graad-(resultaat-)gegevens",
    SLR = "Sluitrecord",
)
tablenames_anbs = dict(
    VLP = "Voorlooprecord",
    BLB = "Bekostigingsloopbaan student",
    BRD = "Bekostigingsresultaat deelname",
    BRR = "Bekostigingsresultaatresultaat",
    SLR = "Sluitrecord",
)

In [5]:
def process_camelot_output(list_of_tables):
    new_tables = pve_parser.process_list_of_tables(list_of_tables)
    new_tables = [pve_parser.extend_table(df) for df in new_tables]
    new_tables = [
        df.assign(verplicht=pve_parser.boolify(df.verplicht))
        if 'verplicht' in df.columns else df
        for df in new_tables
    ]
    return new_tables

def create_json_data(tables, tablenames, eigenschappen):
    new_tables = process_camelot_output(tables)
    velden = {k.lower():new_tables[i].to_dict(orient='index') for i,k in enumerate(tablenames.keys())}

    data = dict(
        eigenschappen = eigenschappen,
        recordsoorten = {k.lower():v for k,v in tablenames.items()},
        velden = velden,
    )
    return data

In [13]:
tables_anbs[2].df.iloc[0,2] = 'Verplicht'
tables_anbs[2].df.iloc[0,3] = 'Definitie/toelichting'

In [14]:
json_data = create_json_data(tables_anbs, tablenames_anbs, eig_anbs)
# json_data = create_json_data(tables_obov, tablenames_obov, eig_obo)

In [15]:
file = 'tabledefs.anbs.1.json'
# file = 'tabledefs.obov.1.json'
with open(file, 'w', encoding='utf8') as f:
    f.write(json.dumps(json_data, indent=4, ))